In [ ]:
!pip install -q -U bitsandbytes wandb
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepar

In [ ]:
# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from datasets import load_dataset
import transformers
from huggingface_hub import notebook_login
from tqdm import tqdm
import json

In [ ]:
notebook_login()

In [ ]:
# Load in Llama-2-7b model using BitsAndBytes

model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Load in the data

full_data = load_dataset("jackhogan/agemo_json_prompts")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5319 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/591 [00:00<?, ? examples/s]

In [ ]:
# Load fine-tune
peft_model = PeftModel.from_pretrained(model, "jackhogan/llama_agemo_finetune")

In [ ]:
def json_out(scrambled_json):
  prompt = """
  Below is a JSON string containing a syntactic error. Return the corrected JSON string.\n\n### Broken JSON:\n{}\n\n### Repaired JSON:\n
  """.format(scrambled_json)

  inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
  prompt_len = inputs['input_ids'].shape[1]
  output_tokens = peft_model.generate(**inputs, max_new_tokens=200)

  return '{' + '{'.join(tokenizer.decode(output_tokens.squeeze()[prompt_len:-1]).split('{')[1:])

In [ ]:
prompt = """
Below is a JSON string containing a syntactic error. Return the corrected JSON string.\n\n### Broken JSON:\n{}\n\n### Repaired JSON:\n
""".format(full_data['test'][invalid[0]]['scrambled'])

inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
prompt_len = inputs['input_ids'].shape[1]
output_tokens = peft_model.generate(**inputs, max_new_tokens=200)

In [ ]:
passed = []
invalid = []
failed = []

for i in tqdm(range(len(full_data['test']))):
  test_case = full_data['test'][i]['scrambled']
  correct = json.loads(full_data['test'][i]['correct'])

  try:
    # Try to load the JSON
    unscrambled = json.loads(json_out(test_case))
    if unscrambled == correct:
      passed.append(i)
    else:
      failed.append(i)
  except ValueError as e:
    invalid.append(i)

100%|██████████| 591/591 [1:32:32<00:00,  9.40s/it]


In [ ]:
print("Successfully unscrambled: {:.2f}%".format(100*len(passed)/len(full_data['test'])))
print("Valid JSON but non-matching: {:.2f}%".format(100*len(failed)/len(full_data['test'])))
print("Invalid JSON: {:.2f}%".format(100*len(invalid)/len(full_data['test'])))

Successfully unscrambled: 98.98%
Valid JSON but non-matching: 0.51%
Invalid JSON: 0.51%


In [ ]:
len(invalid)

3

In [ ]:
# Check example of failed JSON
test_case = full_data['test'][failed[0]]['scrambled']
correct = json.loads(full_data['test'][failed[0]]['correct'])

print(correct)
print(json_out(test_case))


{'albumId': 74, 'id': 3695, 'title': 'totam expedita ut est id sunt nostrum dolorem', 'url': 'https://via.placeholder.com/600/d8600c', 'thumbnailUrl': 'https://via.placeholder.com/150/d8600c'}
{"albumId": 74, "id": 3695, "title": "totam expedita ut est id nostrum dolorem", "url": "https://via.placeholder.com/600/d8600c", "thumbnailUrl": "https://via.placeholder.com/150/d8600c"}


In [ ]:
# Check example of invalid JSON
test_case = full_data['test'][invalid[0]]['scrambled']
correct = json.loads(full_data['test'][invalid[0]]['correct'])

print(correct)
print(json_out(test_case))

{'albumId': {'albumId': {'albumId': 77, 'id': 3821, 'title': 'excepturi consequatur praesentium et ea quis', 'url': 'https://via.placeholder.com/600/d86df6', 'thumbnailUrl': 'https://via.placeholder.com/150/d86df6'}, 'id': 2587, 'title': 'ducimus aut ullam', 'url': 'https://via.placeholder.com/600/593c3', 'thumbnailUrl': 'https://via.placeholder.com/150/593c3'}, 'id': 2040, 'title': 'sequi ut sint non aut', 'url': 'https://via.placeholder.com/600/c000de', 'thumbnailUrl': 'https://via.placeholder.com/150/c000de'}
{"albumId": {"albumId": {"albumId": 77, "id": 3821, "title": "excepturi consequatur praesentium et ea quis", "url": "https://via.placeholder.com/600/d86df6", "thumbnailUrl": "https://via.placeholder.com/150/d86df6"}, "id": 2587, "title": "ducimus aut ullam", "url": "https://via.placeholder.com/600/593c3", "thumbnailUrl": "https://via.placeholder.com/150/593c3"}, "id": 2040, "title": "sequi ut sint non aut", "url": "https://via.placeholder.com/600/c000de",
